In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD

from sklearn.metrics import accuracy_score

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

## Preparing data

In [2]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.4,
        zoom_range=0.4,
        horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train', target_size=(150, 150), batch_size=batch_size, class_mode='binary', shuffle=False)
test_generator = test_datagen.flow_from_directory('test', target_size=(150, 150), batch_size=batch_size, class_mode='binary', shuffle=False)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [3]:
train_labels = []

for image, label in train_generator:
    if len(train_labels) == 498:
        break
    train_labels += list(label)
    
test_labels = []

for image, label in test_generator:
    if len(test_labels) == 500:
        break
    test_labels += list(label)

In [4]:
len(train_labels), len(test_labels)

(498, 500)

# Modelling

## Simple model

In [26]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [27]:
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=8
)

Epoch 1/8
100/100 [==============================] - 66s 661ms/step - loss: 0.7902 - accuracy: 0.5558
Epoch 2/8
100/100 [==============================] - 65s 653ms/step - loss: 0.6924 - accuracy: 0.5873
Epoch 3/8
100/100 [==============================] - 65s 651ms/step - loss: 0.7035 - accuracy: 0.5834
Epoch 4/8
100/100 [==============================] - 64s 637ms/step - loss: 0.6438 - accuracy: 0.6463
Epoch 5/8
100/100 [==============================] - 64s 639ms/step - loss: 0.6273 - accuracy: 0.6697
Epoch 6/8
100/100 [==============================] - 64s 636ms/step - loss: 0.6121 - accuracy: 0.6900
Epoch 7/8
100/100 [==============================] - 64s 635ms/step - loss: 0.5868 - accuracy: 0.7022
Epoch 8/8
100/100 [==============================] - 62s 618ms/step - loss: 0.5495 - accuracy: 0.7317


In [28]:
predicts = model.predict_generator(test_generator)
accuracy_score(labels, predicts.reshape(-1) >= 0.5)

0.558

## Using pre-trained VGG-16

In [5]:
from tensorflow.keras.applications import VGG16, VGG19

In [6]:
vgg_model = VGG16(include_top=False, weights='imagenet')

In [7]:
vgg_train = list(vgg_model.predict(train_generator))
vgg_test = list(vgg_model.predict(test_generator))

In [8]:
vgg_train_other = np.array([array.reshape(-1) for array in vgg_train])
vgg_test_other = np.array([array.reshape(-1) for array in vgg_test])

In [9]:
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(vgg_train_other, np.array(train_labels), epochs=25, batch_size=batch_size)

Epoch 1/25
498/498 [==============================] - 2s 3ms/sample - loss: 1.3946 - accuracy: 0.5663
Epoch 2/25
498/498 [==============================] - 0s 964us/sample - loss: 0.6680 - accuracy: 0.6888
Epoch 3/25
498/498 [==============================] - 0s 934us/sample - loss: 0.5461 - accuracy: 0.7510
Epoch 4/25
498/498 [==============================] - 1s 1ms/sample - loss: 0.5030 - accuracy: 0.7791
Epoch 5/25
498/498 [==============================] - 1s 1ms/sample - loss: 0.4270 - accuracy: 0.8293
Epoch 6/25
498/498 [==============================] - 1s 1ms/sample - loss: 0.4616 - accuracy: 0.8153
Epoch 7/25
498/498 [==============================] - 1s 1ms/sample - loss: 0.2950 - accuracy: 0.8775
Epoch 8/25
498/498 [==============================] - 0s 948us/sample - loss: 0.2818 - accuracy: 0.8735
Epoch 9/25
498/498 [==============================] - 0s 967us/sample - loss: 0.2763 - accuracy: 0.8976
Epoch 10/25
498/498 [==============================] - 0s 952us/sample - l

In [10]:
predictions = model.predict(vgg_test_other).reshape(-1)
accuracy_score([0] * 250 + [1] * 250, predictions >= 0.5)

0.79

In [11]:
model.save_weights('top_weights.h5')

# Fine-tuning of the top CNN layer

In [23]:
model = VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights('top_weights.h5')

all_layers = model.layers + top_model.layers
model = Sequential(all_layers)

for layer in model.layers[:14]:
    layer.trainable = False

model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])

In [24]:
model.fit_generator(train_generator, steps_per_epoch=100, epochs=10)

Epoch 1/10
100/100 [==============================] - 208s 2s/step - loss: 0.7177 - accuracy: 0.7381
Epoch 2/10
100/100 [==============================] - 207s 2s/step - loss: 0.3497 - accuracy: 0.8447
Epoch 3/10
100/100 [==============================] - 206s 2s/step - loss: 0.2611 - accuracy: 0.8787
Epoch 4/10
100/100 [==============================] - 210s 2s/step - loss: 0.1839 - accuracy: 0.9236
Epoch 5/10
100/100 [==============================] - 206s 2s/step - loss: 0.1495 - accuracy: 0.9333
Epoch 6/10
100/100 [==============================] - 207s 2s/step - loss: 0.1236 - accuracy: 0.9461
Epoch 7/10
100/100 [==============================] - 211s 2s/step - loss: 0.0984 - accuracy: 0.9602
Epoch 8/10
100/100 [==============================] - 206s 2s/step - loss: 0.0640 - accuracy: 0.9782
Epoch 9/10
100/100 [==============================] - 204s 2s/step - loss: 0.0554 - accuracy: 0.9775
Epoch 10/10
100/100 [==============================] - 202s 2s/step - loss: 0.0627 - accura

In [26]:
predicts = model.predict(test_generator)
accuracy_score([0] * 250 + [1] * 250, predicts.reshape(-1) >= 0.5)

0.814